In [1]:
import tqdm
import numpy as np

In [2]:
data = np.load('data.npy')
data = np.reshape(data,[-1,20,28,28])
train_data = data[:1200,:,:,:]
test_data = data[1200:,:,:,:]

In [3]:
def get_minibatch(nb_batch, nb_episode, nb_class, data):
    dshape = data.shape[2:]
    batch_x = np.zeros((nb_batch, nb_episode) + dshape)
    batch_test = np.zeros((nb_batch, nb_episode, 2))
    batch_y = np.zeros((nb_batch, nb_episode), np.int)
    batch_mask = np.ones((nb_batch, nb_episode), np.bool)
    nb_class_total = len(data)
    
    for i in range(nb_batch):
        # classes for learning
        classes = np.random.choice(nb_class_total, nb_class, False)
        
        # index to class
        # class_idx is classes[pinds[i]]
        pidx = np.random.permutation(nb_class)
        
        # sample data
        sample = np.random.randint(0, nb_class, nb_episode)
        batch_y[i] = sample
        
        _, first = np.unique(sample, return_index=True)
        mask = np.ones(nb_episode, np.bool)
        mask[first] = False
        batch_mask[i] = mask
        
        for j in range(nb_class):
            idx = (sample == j)
            eidx = np.random.choice(data.shape[1], np.sum(sample == j), False)
            batch_x[i, idx] = data[classes[pidx[j]], eidx]
            batch_test[i, idx, 0] = classes[pidx[j]]
            batch_test[i, idx, 1] = eidx
            
        return batch_x, batch_y, batch_mask, batch_test

In [4]:
nb_nb_batch = 1
nb_episode = 32
nb_class = 5

for iter in tqdm.tqdm(range(10000)):
    batch_x, batch_y, batch_m, batch_test = get_minibatch(nb_nb_batch, nb_episode, nb_class, train_data)

    # test
    for i in range(nb_class):
        idx = batch_y == i
        if np.sum(idx) > 0:
            t = batch_test[idx][:,0]
            assert(((t - t[0]) == 0).all())        
            assert(len(np.unique(batch_test[idx][:,1])) == len(batch_test[idx][:,1]))
            
            fidx = np.where(batch_y==i)[1][0]
            ridx = np.where(batch_y==i)[1][1:]
            assert(batch_m[0][fidx] == False)
            assert(batch_m[0][ridx].all())

100%|██████████| 10000/10000 [00:09<00:00, 1088.72it/s]


In [5]:
print("test passed!!")

test passed!!


In [6]:
batch_x, batch_y, batch_m, _ = get_minibatch(nb_nb_batch, nb_episode, nb_class, train_data)